<a href="https://colab.research.google.com/github/draglar/Moringa-dsc6/blob/master/Moringa_Data_Science_Prep_W2_Independent_Project_2019_06_Ian_Muigai_SQL_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### loading the extension and opening the files


In [ ]:
%load_ext sql
%sql sqlite://
import csv
import pandas as pd

In [ ]:
with open('Population_by_state.csv','r') as f:
  population=pd.read_csv(f,index_col=0,encoding='uft-8')
%sql drop table if exists population 
%sql persist population
%sql select * from population order by State limit 5

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
ALABAMA,4858979
ALASKA,738432
AMERICAN SAMOA,54343
ARIZONA,6828065
ARKANSAS,2978204


In [ ]:
with open('GrandElectors_by_state.csv','r') as e:
  electors=pd.read_csv(e,index_col=0,encoding='uft-8')
%sql drop table if exists electors
%sql persist electors
%sql select * from electors order by State limit 5

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


### 1:To join the 2 tables:
You notice States are not capitalized the same way in both tables (one is in uppercase letters, the other not), so you will first need to convert all to uppercase, for instance.
Now you can join the tables on the state key.

lower case is just my preference because its easier to read

In [ ]:
%%sql
update population
set State=lower(State);
update electors
set State=lower(State);

 * sqlite://
56 rows affected.
51 rows affected.


[]

### 2: change the name of the "District of Columbia" state to its short version "DC".

In [ ]:
%%sql
update population
set State="dc" where State="district of columbia";
update electors
set State="dc" where State="district of columbia"

 * sqlite://
1 rows affected.
1 rows affected.


[]

### 3:To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.

ratio== 1 elector represents x number of people in a State

In [ ]:
%sql alter table population add ratio

 * sqlite://
Done.


[]

In [ ]:
%%sql
update population
set ratio=(select Population/GrandElectors from electors where electors.State=population.State)

 * sqlite://
56 rows affected.


[]

since the only unique column from  the GrandElectors_by_state.csv is Grandelectors i updated the Population_by_state.csv with the values for ease of use and to reduce the size of code needed to solve the task

In [ ]:
%sql alter table population add Electors

 * sqlite://
Done.


[]

populating the column

In [ ]:
%%sql
update population
set Electors=(select GrandElectors from electors where electors.State=population.State)

 * sqlite://
56 rows affected.


[]

An annomaly, these dont have a ratio because they dont appear in GrandElectors_by_state.csv indicating they dont have 
electors

In [ ]:
%%sql
select State from population
where not exists (select State from electors where population.State=electors.State)

 * sqlite://
Done.


State
american samoa
guam
northern mariana islands
puerto rico
u.s. virgin islands


### 4: order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.

That is ratio becomes the priority when ordering lists

In [ ]:
%sql select * from population order by ratio desc

 * sqlite://
Done.


State,Population,ratio,Electors
texas,27469114,722871,38
california,39144818,711723,55
florida,20271272,699009,29
new york,19795791,682613,29
north carolina,10042802,669520,15
ohio,11613423,645190,18
virginia,8382993,644845,13
illinois,12859995,642999,20
pennsylvania,12802503,640125,20
new jersey,8958013,639858,14


### 5: compute the running total of Grand Electors in that sorted list

In [ ]:
%%sql
select *,(select sum(Electors) from population t1 where t1.ratio>=t2.ratio order by ratio desc) as running_total from population t2
order by ratio desc

 * sqlite://
Done.


State,Population,ratio,Electors,running_total
texas,27469114,722871,38,38
california,39144818,711723,55,93
florida,20271272,699009,29,122
new york,19795791,682613,29,151
north carolina,10042802,669520,15,166
ohio,11613423,645190,18,184
virginia,8382993,644845,13,197
illinois,12859995,642999,20,217
pennsylvania,12802503,640125,20,237
new jersey,8958013,639858,14,251


### 6: compute the half of the total of Grand Electors overall (in the whole country)

In [ ]:
%sql select sum(Electors)/2 as half_Grand_Electors from population

 * sqlite://
Done.


half_Grand_Electors
269


### 7: filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.

In [ ]:
%%sql
select State from population t2
where (select sum(Electors) from population t1 where t1.ratio>=t2.ratio order by ratio desc)<=(select sum(Electors)/2 as half_Grand_Electors from population)
order by ratio desc

 * sqlite://
Done.


State
texas
california
florida
new york
north carolina
ohio
virginia
illinois
pennsylvania
new jersey


-Add the first state for which the running total is larger than the threshold.

In [ ]:
%%sql
select State from population t2
where (select sum(Electors) from population t1 where t1.ratio>=t2.ratio order by ratio desc)>=(select sum(Electors)/2 as half_Grand_Electors from population)
order by ratio desc limit 1

 * sqlite://
Done.


State
arizona


In conclusion the states that would be best to invest in are:
texas,california,florida,new york,north carolina,ohio,
virginia,illinois,pennsylvania,new jersey,georgia and arizona, based on their cummulative running totals.